In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sqlalchemy.orm import sessionmaker, Session
from sqlalchemy import create_engine, select, asc, desc, func
import logging
import time
from TickerScrape.models import Security, AssetClass, Country, Currency, Industry, Exchange, Tag

In [3]:
# uri = 'sqlite:///databases/TickerScrape.db'
uri = 'sqlite:////Users/zenman618/Documents/git_packages/VisualStudioGit/TickerScrape/sqlite_files/TickerScrape.db'

In [4]:
# df = pd.read_sql_table("security", uri, schema=None, index_col='id', coerce_float=True, chunksize=None, columns=['ticker, name']) # columns=['ticker, name'], parse_dates='Inception_date',
# df.head()

In [5]:
def init_engine(uri):
    logging.basicConfig()
    logging.getLogger('sqlalchemy.engine').setLevel(logging.INFO)
    engine = create_engine(uri)
    return engine

def db_connect(engine):
    connection = engine.connect()
    logging.info("****_Ticker_Pipeline: database connected****")
    return connection

def db_session(engine):
    # Session = sessionmaker(bind=engine)
    # session = Session()
    session = Session(engine)
    logging.info("****_Ticker_Pipeline: database connected****")
    return session

In [6]:
engine = init_engine(uri)

In [ ]:
# %%timeit
with db_connect(engine) as con:
    result = con.execute("select ticker from security")
    # for row in result:
    #     print("ticker:", row['ticker'])
    df = pd.DataFrame(result)
df.tail()

In [17]:
# %%timeit
with db_session(engine) as session:
    result = session.execute("select ticker, name from security")
    df = pd.DataFrame(result, columns = ['Ticker', 'Name'])
df.tail()

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:select ticker, name from security
INFO:sqlalchemy.engine.Engine:[cached since 103.2s ago] ()
INFO:sqlalchemy.engine.Engine:ROLLBACK


,Ticker,Name
80584,TACO,Del Taco Restaurants Inc.
80585,PEFDF,Defli Ltd
80586,DNCVF,Defiance Silver Corp.
80587,DEFTF,DeFi Technologies Inc.
80588,DFTC,Defentect Group Inc.


In [26]:
# %%timeit
with db_session(engine) as session:
    result = session.execute("select ticker, name from security")
    # print(type(result))
    # for dict_row in result.mappings():
    #     ticker = dict_row['ticker']
    #     name = dict_row['name']
    #     print (dict_row)

    df = pd.DataFrame(result, columns = ['Ticker', 'Name'])
df.tail()

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:select ticker, name from security
INFO:sqlalchemy.engine.Engine:[cached since 863.2s ago] ()
INFO:sqlalchemy.engine.Engine:ROLLBACK


,Ticker,Name
80584,TACO,Del Taco Restaurants Inc.
80585,PEFDF,Defli Ltd
80586,DNCVF,Defiance Silver Corp.
80587,DEFTF,DeFi Technologies Inc.
80588,DFTC,Defentect Group Inc.


In [29]:
def all_tickers():
    with db_session(engine) as session:
        result = session.execute("select ticker, name from security")
        df = pd.DataFrame(result, columns = ['Ticker', 'Name'])
        df['Label'] = df['Ticker'] + " (" + df['Name'] + ")"
        symbols = df['Ticker']
        labels = df['Label']
    return labels, symbols

labels, symbols = all_tickers()

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:select ticker, name from security
INFO:sqlalchemy.engine.Engine:[cached since 1468s ago] ()
INFO:sqlalchemy.engine.Engine:ROLLBACK


In [30]:
options=[{'label': x, 'value': y} for x, y in zip(labels, symbols)]

options

[{'label': 'ACAZF (Acadian Timber Corp.)', 'value': 'ACAZF'},
 {'label': 'AKR (Acadia Realty Trust)', 'value': 'AKR'},
 {'label': 'ACAD (ACADIA Pharmaceuticals Inc.)', 'value': 'ACAD'},
 {'label': 'ACHC (Acadia Healthcare Co. Inc.)', 'value': 'ACHC'},
 {'label': 'ASO (Academy Sports & Outdoors Inc.)', 'value': 'ASO'},
 {'label': 'ACTG (Acacia Research Corp. - Acacia Technologies)',
  'value': 'ACTG'},
 {'label': 'ACPGF (Acacia Pharma Group PLC)', 'value': 'ACPGF'},
 {'label': 'ACCA (Acacia Diversified Holdings Inc.)', 'value': 'ACCA'},
 {'label': 'ACIU (AC Immune S.A.)', 'value': 'ACIU'},
 {'label': 'ABVC (ABVC BioPharma Inc.)', 'value': 'ABVC'},
 {'label': 'AHFI (Absolute Health & Fitness Inc.)', 'value': 'AHFI'},
 {'label': 'ABST (Absolute Software Corp.)', 'value': 'ABST'},
 {'label': 'ASCN (Absecon Bancorp)', 'value': 'ASCN'},
 {'label': 'ABSI (Absci Corp.)', 'value': 'ABSI'},
 {'label': 'AGRPY (Absa Group Ltd.)', 'value': 'AGRPY'},
 {'label': 'ASPAW (Abri SPAC I Inc. Wt)', 'value'

In [28]:
df['Label'] = df['Ticker'] + " (" + df['Name'] + ")"
symbols = df['Ticker']
labels = df['Label']
df

,Ticker,Name,Label
0,ACAZF,Acadian Timber Corp.,ACAZF (Acadian Timber Corp.)
1,AKR,Acadia Realty Trust,AKR (Acadia Realty Trust)
2,ACAD,ACADIA Pharmaceuticals Inc.,ACAD (ACADIA Pharmaceuticals Inc.)
3,ACHC,Acadia Healthcare Co. Inc.,ACHC (Acadia Healthcare Co. Inc.)
4,ASO,Academy Sports & Outdoors Inc.,ASO (Academy Sports & Outdoors Inc.)
...,...,...,...
80584,TACO,Del Taco Restaurants Inc.,TACO (Del Taco Restaurants Inc.)
80585,PEFDF,Defli Ltd,PEFDF (Defli Ltd)
80586,DNCVF,Defiance Silver Corp.,DNCVF (Defiance Silver Corp.)
80587,DEFTF,DeFi Technologies Inc.,DEFTF (DeFi Technologies Inc.)


In [10]:
sltd_sec = 'AAPL'
# %%timeit
with db_session(engine) as session:
    stmt = select(Security.id, Security.ticker, Security.name).where(Security.ticker == sltd_sec).order_by(Security.ticker)
    result = session.execute(stmt)
    df = pd.DataFrame(result, columns = ['Sec_Id', 'Ticker', 'Name'])
df.tail()

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT security.id, security.ticker, security.name 
FROM security 
WHERE security.ticker = ? ORDER BY security.ticker
INFO:sqlalchemy.engine.Engine:[cached since 220.5s ago] ('AAPL',)
INFO:sqlalchemy.engine.Engine:ROLLBACK


,Sec_Id,Ticker,Name
0,22758,AAPL,Leverage Shares 1X Apple ETP


In [16]:
# %%timeit
with db_session(engine) as session:
    result = session.execute(select(Security.ticker, Security.name).order_by(Security.ticker))
    df = pd.DataFrame(result, columns = ['Ticker', 'Name'])
df.tail()

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT security.ticker, security.name 
FROM security ORDER BY security.ticker
INFO:sqlalchemy.engine.Engine:[cached since 85.6s ago] ()
INFO:sqlalchemy.engine.Engine:ROLLBACK


,Ticker,Name
80584,ZZE.H,Zidane Capital Corp.
80585,ZZMS,Commerzbank AG ADR
80586,ZZZ,Sleep Country Canada Holdings Inc.
80587,ZZZD,BMO Tactical Dividend ETF Fund
80588,ZZZOF,Zinc One Resources Inc.


In [ ]:
df.head()

In [ ]:
with db_session(engine) as session:
    result = session.execute(select(Security.id, Security.ticker).order_by(Security.ticker))
    res = result.fetchone()
    print(res)
    # df = pd.DataFrame(result, columns = ['Sec_Id','Ticker'])
df.tail()

In [ ]:
with db_session(engine) as session:
    # result = session.execute(select(Security.name, Country.name).join(Security.name).order_by(Security.id, Country.id))
    country = session.execute(select(Country).where(Country.name == "United States"))
    country = Country(name="United States")
    # country = session.query(Country).filter(Country.name=="United States")
    # print (country)
    print (country)
    result = session.execute(select(Security.ticker).where(Security.countries.contains(country)))
    df = pd.DataFrame(result)

In [ ]:
sel_country = "United States"

with db_session(engine) as session:
    stmt = (select(Security.id, Security.ticker, Security.name).where(Security.countries.any(Country.name == sel_country)))
    by_country = session.execute(stmt).all()
    df = pd.DataFrame(by_country, columns = ['Sec_Id', 'Name', 'Ticker'])
print(df.Ticker.count())
df.tail()

In [ ]:
sel_asset = "Stocks"

with db_session(engine) as session:
    stmt = (select(Security.id, Security.ticker, Security.name).where(Security.asset_class_id == (select(AssetClass.id).where(AssetClass.name == sel_asset))))
    by_asset = session.execute(stmt).all()
    df = pd.DataFrame(by_asset, columns = ['Sec_Id', 'Name', 'Ticker'])
print(df.Ticker.count())
df.tail()

In [ ]:
sel_industry = "Software"

with db_session(engine) as session:
    stmt = (select(Security.id, Security.ticker, Security.name).where(Security.industries.any(Industry.name == sel_industry)))
    by_industry = session.execute(stmt).all()
    df = pd.DataFrame(by_industry, columns = ['Sec_Id', 'Name', 'Ticker'])
print(df.Ticker.count())
df.tail()